In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn.functional as F
from tqdm import tqdm
from cnn_models import LeNet
from PCLeNet import PCLeNet, PCLeNetv3, PCLeNetv3Deep
from PCLeNet_L1MoreFeatures import PCLeNetv2
from CustomDataset import PreloadedDataset
from train_loops import train, train_pc_classification, train_pc_error, train_pc_ec
from my_funcs import RandomGaussianNoise, Scale
import matplotlib.pyplot as plt

c:\Users\Joe\anaconda3\envs\ml-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# Your code here!
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    RandomGaussianNoise(std=0.001),
    transforms.RandomAffine(degrees=35, translate=(0.15,0.15), scale=(0.90,1.10)),
    ])
dataset = PreloadedDataset('..\\..\\..\\datasets\\TinyImageNet30\\train_set\\train_set', 
                         (0,3,64,64), 
                         transform=transform,
                        )
CLASSES = dataset.classes
NUM_CLASSES = len(CLASSES)
INPUT_SHAPE = dataset[0][0].shape
VAL_RATIO = 0.2
train_dataset, val_dataset = dataset.cross_val_split_by_class(VAL_RATIO, 0, device=device)

In [12]:
seed = 42
torch.manual_seed(seed)
model_name = "LeNet"
LEARNING_RATE = 3e-4
lenet = LeNet(INPUT_SHAPE[0], NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(lenet.parameters(), lr=LEARNING_RATE)
cnn_train_loss, cnn_train_acc, cnn_val_loss, cnn_val_acc, _ = train(
    lenet,
    train_dataset,
    val_dataset,
    optimiser,
    criterion,
    model_name,
    500,
    batch_size=300,
    save_model=False,
    model_dir="TIN/models/",
    log_dir="TIN/logs/",
    device=device,
)

In [9]:
seed = 41
torch.manual_seed(seed)
#  Standard LeNet CNN
model_name = "PCLeNet_ec_relu"
LEARNING_RATE = 3e-6
lenet = PCLeNet(INPUT_SHAPE, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(lenet.parameters(), lr=LEARNING_RATE)
step = train_pc_ec(
    lenet,
    train_dataset,
    val_dataset,
    criterion,
    model_name,
    500,
    batch_size=300,
    pc_learning_rate=LEARNING_RATE,
    log_dir="TIN/logs",
    model_dir="TIN/models",
    save_model=False,
    device=device,
    # step=step,
)

KeyboardInterrupt: 

In [ ]:

seed = 42
torch.manual_seed(seed)
#  Standard LeNet CNN
model_name = "PCLeNet_ec"
LEARNING_RATE = 3e-4
lenet = PCLeNet(INPUT_SHAPE, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(lenet.parameters(), lr=LEARNING_RATE)
step = train_pc_ec(
    lenet,
    train_dataset,
    val_dataset,
    criterion,
    model_name,
    500,
    batch_size=300,
    log_dir="TIN/logs",
    model_dir="TIN/models",
    save_model=False,
    device=device,
    step=step,
)

In [7]:

torch.manual_seed(0)
#  Standard LeNet CNN
model_name = "PCLeNet_classification_WD3e-2"
LEARNING_RATE = 3e-4
lenet = PCLeNet(INPUT_SHAPE, NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(lenet.parameters(), lr=LEARNING_RATE, weight_decay=3e-2)
step = train_pc_classification(
    lenet,
    train_dataset,
    val_dataset,
    optimiser,
    criterion,
    model_name,
    500,
    batch_size=300,
    log_dir="TIN",
    save_model=False,
    device=device,
    plot_errs=True,
)

KeyboardInterrupt: 